In [1]:
!pip install -U transformers accelerate bitsandbytes

/bin/bash: /media/rdi/043764CF611486F91/NLP/llm-zoomcamp/venv/bin/pip: /media/rdi/043764CF611486F9/NLP/llm-zoomcamp/venv/bin/python3.10: bad interpreter: No such file or directory


In [15]:
import os
import requests 
import minsearch
# import torch
# from transformers import AutoModelForCausalLM
# from transformers import AutoTokenizer
# from huggingface_hub import login
# from transformers import pipeline

os.environ['HF_HOME'] = '/run/cache/'

In [16]:

docs_url = 'https://github.com/DataTalksClub/llm-zoomcamp/blob/main/01-intro/documents.json?raw=1'
docs_response = requests.get(docs_url)
documents_raw = docs_response.json()

documents = []

for course in documents_raw:
    course_name = course['course']

    for doc in course['documents']:
        doc['course'] = course_name
        documents.append(doc)

index = minsearch.Index(
    text_fields=["question", "text", "section"],
    keyword_fields=["course"]
)

index.fit(documents)

In [17]:
def search(query):
    boost = {'question': 3.0, 'section': 0.5}

    results = index.search(
        query=query,
        filter_dict={'course': 'data-engineering-zoomcamp'},
        boost_dict=boost,
        num_results=5
    )

    return results

In [18]:
def build_prompt(query, search_results):
    prompt_template = """
You're a course teaching assistant. Answer the QUESTION based on the CONTEXT from the FAQ database.
Use only the facts from the CONTEXT when answering the QUESTION.

QUESTION: {question}

CONTEXT: 
{context}
""".strip()

    context = ""
    
    for doc in search_results:
        context = context + f"section: {doc['section']}\nquestion: {doc['question']}\nanswer: {doc['text']}\n\n"
    
    prompt = prompt_template.format(question=query, context=context).strip()
    return prompt

def llm(prompt):
    response = client.chat.completions.create(
        model='gemma:2b',
        messages=[{"role": "user", "content": prompt}]
    )
    
    return response.choices[0].message.content

In [19]:
def rag(query):
    search_results = search(query)
    prompt = build_prompt(query, search_results)
    answer = llm(prompt)
    return answer

In [22]:
from openai import OpenAI

client = OpenAI(
    base_url='http://localhost:11434/v1/',
    api_key='ollama',
)

In [23]:
llm(' "10 * 10". Whats the answer?')


'The answer is 100.\n\n10 * 10 = 100.'

In [43]:
def llm(prompt):
    response = client.chat.completions.create(
        model='gemma:2b',
        messages=[{"role": "user", "content": prompt}],
        temperature=0.0
    )
    
    return response

In [44]:
prompt = "What's the formula for energy?"
response =llm(prompt)

In [45]:


# Count the number of tokens (split by whitespace here, adapt as needed)
print(f"Number of output tokens: {response.usage.completion_tokens}")


Number of output tokens: 304
